In [0]:
!pip install gensim
!pip install bs4
!pip install cloudscraper

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-67c4fcee-9a32-4aa2-b660-c2c01d8d3b27/bin/python -m pip install --upgrade pip' command.
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-67c4fcee-9a32-4aa2-b660-c2c01d8d3b27/bin/python -m pip install --upgrade pip' command.
  Using cached cloudscraper-1.2.71-py2.py3-none-any.whl (99 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-67c4fcee-9a32-4aa2-b660-c2c01d8d3b27/bin/python -m pip install --upgrade pip' command.
  Using cached google_generativeai-0.4.1-py3-none-any.whl (137 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl (189 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl (13

In [0]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import size, col, explode, expr, sum, first, udf, split, when
import cloudscraper
from bs4 import BeautifulSoup
import pathlib
import textwrap
from pyspark.sql.functions import udf
import numpy as np

spark = SparkSession.builder.getOrCreate()

In [0]:
people = spark.read.parquet('/linkedin/people')
people = people.select("about", "id",'certifications','education','experience','followers','position','posts','recommendations' ,'url').\
where(size("posts")>0)
people = people.sample(withReplacement=False, fraction=2500/people.count(), seed=42)


In [0]:
def convert_experience_to_duration(duration):
    total_lst = []
    for dur in duration:
        total = 0
        if dur is None:
            continue

        unit_to_months = {'year': 12, 'years': 12, 'month': 1, 'months': 1}

        if 'less than a year' in dur:
            total += 6
        else:
            total_months = 0
            parts = dur.split()
            for i in range(0, len(parts), 2):
                value = int(parts[i])
                unit = parts[i + 1]
                total_months += value * unit_to_months.get(unit, 0)  # Use .get() for safety

            total += total_months
        total_lst.append(total)
    
    if len(total_lst) == 0:
        return 0
    sum_total = 0
    for elem in total_lst:
        sum_total += elem
    return sum_total

convert_experience_to_duration_udf = udf(convert_experience_to_duration, returnType=IntegerType())

@udf(ArrayType(StringType()))
def extract_text_from_links(links):
    all_text = []
    scraper = cloudscraper.create_scraper()

    for link in links:
        match_response = scraper.get(link)
        html_content = match_response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        
        try:
            text = soup.find('div', class_='article-main__content max-w-[744px]').text
            all_text.append(text)
        except:
            try:
                text = soup.find('div', class_='article-main__content').text
                all_text.append(text)
            except:
                try:
                    text = soup.find('div', class_='article-content-blocks').text
                    all_text.append(text)
                except:
                    continue
    
    return all_text

@udf(ArrayType(StringType()))
def extract_title(links):
    all_titles = []
    scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance

    for link in links:
        match_response = scraper.get(link)
        html_content = match_response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        try:
            text = soup.find('title').text
            all_titles.append(text)
        except:
            continue
    return all_titles

@udf(StructType([
    StructField("comments", MapType(IntegerType(), ArrayType(StringType()))),
    StructField("total_comments", IntegerType())
]))
def get_comments(links):
    comments_dict = {}
    total_comments = 0    
    for i, link in enumerate(links):
        comments = []
        scraper = cloudscraper.create_scraper()
        match_response = scraper.get(link)
        html_content = match_response.text        
        soup = BeautifulSoup(html_content, 'html.parser')        
        comment_blocks = soup.find_all('section', class_='comment')
        for block in comment_blocks:
            comment_text_element = block.find('p', class_='comment__text')
            if comment_text_element:
                comment_text = comment_text_element.text.strip()
                comments.append(comment_text)
                total_comments += 1
        comments_dict[i] = comments
    return comments_dict, total_comments

In [0]:
people= people.\
withColumn("degree", col("education").getItem("degree")).\
withColumn("field", col("education").getItem("field")).\
drop("education").\
withColumn("duration_short", col("experience").getItem("duration_short")).\
drop("experience").\
withColumn('duration_short_numeric', convert_experience_to_duration_udf(col('duration_short'))).\
drop('duration_short').\
withColumn('certifications', expr("transform(certifications, x -> x.title)")).\
withColumn('posts', expr("transform(posts, x -> x.link)")).\
withColumn("post_content", extract_text_from_links(col("posts"))).\
withColumn("post_title", extract_title(col("posts"))).\
withColumn("comments_and_counts", get_comments(col("posts"))).\
withColumn("comments", col("comments_and_counts").comments).\
withColumn("total_comments", col("comments_and_counts").total_comments).\
drop("comments_and_counts")

In [0]:
people.write.mode("overwrite").format("json").save("/Workspace/Users/shir.g@campus.technion.ac.il/final_project/people.json")
people = spark.read.format("json").load("/Workspace/Users/shir.g@campus.technion.ac.il/final_project/people.json")
people.display()